In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('data_lemm.csv')
# data.dropna(inplace=True)
# data.reset_index(drop=True, inplace=True)
# data.replace('\d+', '', regex=True, inplace=True)
# data.replace("^a-zA-Z0-9]+", "", regex=True, inplace=True)

In [ ]:
# data['class'] = 0
# data.loc[data['Category'] == 'Политика', 'class'] = 1
# data.loc[data['Category'] == 'Экономика', 'class'] = 2
# data.loc[data['Category'] == 'Наука', 'class'] = 3
# data.loc[data['Category'] == 'Статьи', 'class'] = 4
# data.loc[data['Category'] == 'Книги', 'class'] = 5



In [ ]:
data.to_csv('./data_lemm.csv')

In [ ]:


def tfidf(texts):

    corpus = texts.to_numpy()
    
    vocabulary = list(set([item for sublist in list(data['Text'].astype(str)) for item in sublist.split()]))
    pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                    ('tfid', TfidfTransformer())]).fit(corpus)
    pipe['count'].transform(corpus).toarray()
    pipe['tfid'].idf_

    return pipe.transform(corpus)

test = tfidf(data['Text'].astype(str))

In [ ]:
test

<9096x48001 sparse matrix of type '<class 'numpy.float64'>'
	with 993973 stored elements in Compressed Sparse Row format>

In [ ]:
def corpuses_embedding_count_vectorize(texts):

    corpus = texts.to_numpy()
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)

    return X.toarray()

test = corpuses_embedding_count_vectorize(data['Text'].astype(str))


In [ ]:
# [item for sublist in list(data['Text'].astype(str).values) for item in sublist.split()]

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

def doc_to_vec(texts):
    #tokenize and tag the card text
    card_docs = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(data['Text'].astype(str))]

    model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 40)
    #build vocab
    model.build_vocab(card_docs)
    #train model
    model.train(card_docs, total_examples=model.corpus_count
                , epochs=model.epochs)

    card2vec = [model.infer_vector((texts[i].split(' '))) 
            for i in range(0,len(texts))]
    
    dtv= np.array(card2vec).tolist()

    return dtv

doc_to_vec(data['Text'])


Fast Text не надо

In [ ]:
from gensim.models.fasttext import FastText

def ft(texts):


    embedding_size = 60
    window_size = 40
    min_word = 5
    down_sampling = 1e-2

    ft_model = FastText(texts,
                        size=embedding_size,
                        window=window_size,
                        min_count=min_word,
                        sample=down_sampling,
                        sg=1,
                        iter=1)
    
    return np.array([[ft_model.wv[word] for word in text] for text in data['Text']])
